## Let's start by setting up the environment.

In [ ]:
import sys
from os import path
sys.path.append("../")
import os
import im

os.environ['AWS_DEFAULT_REGION']='us-west-2'

ims = im.Session()

# Replace with a role that gives IM access to s3 and cloudwatch
role='IMRole'

## Downloading test and training data

In [ ]:
import utils

utils.cifar10_download()

## Uploading the data

In [ ]:
inputs = ims.upload_data(path='/tmp/cifar10_data', key_prefix='data/cifar10')

# Complete source code

In [ ]:
!tree

# Running TensorFlow training on IM

In [ ]:
from im.tensorflow import TensorFlow


sorce_dir = os.path.join(os.getcwd(), 'source_dir')
estimator = TensorFlow(entry_point='resnet_cifar_10.py',
                       source_dir=sorce_dir,
                       role=role,
                       hyperparameters={'training_steps': 1000, 'evaluation_steps': 100},
                       train_instance_count=2, train_instance_type='ml.p2.xlarge')

estimator.fit(inputs)

# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint which serves prediction requests in real-time.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge')

# Deleting the endpoint

In [ ]:
estimator.delete_endpoint()